# 共役勾配法で連立方程式の解を計算しよう

共役勾配法とは，連立方程式の解を，コスト関数の最小化問題に落とし込むことで計算する手法である．
以下の方程式を解くことを考える．
\begin{eqnarray}
    Ax = b
\end{eqnarray}
ただし，$A$は対称な正定値行列である．

このとき，連立方程式の解は，
\begin{eqnarray}
    f(x) = \frac{1}{2}x^TAx - b^T x
\end{eqnarray}
を最小化する$x$と等しくなる．これは，$A$が対称行列のとき，
\begin{eqnarray}
    \frac{d f(x)}{dx_i} = a_{ii}x_i + \frac{1}{2}\sum_{k\neq i}^{}(a_{ik} + a_{ki}{})x_k - b = (Ax-b)_ i = 0\\
    \therefore \frac{d f(x)}{dx} = Ax-b = 0
\end{eqnarray}
が$f(x)$の最適化条件だからである．

> $A$が正定値でないと，$Ax-b=0$が$f(x)$の最適化条件でなくなる．

## アルゴリズムの手順
$x_k$を$k$回目の(後に説明する)イテレーションで求めた解$x$とする．


$x_0=0$を準備する．$k=0$から次の操作を$x_{k}$が収束するまでイテレーションを繰り返すことで，解を計算する．
\begin{eqnarray}
    r_k = b-Ax_k
\end{eqnarray}
\begin{eqnarray}
    p_{k} = r_k  - \sum_{i=0}^{k-1} \frac{p_i^T A r_k}{p_i A p_i} p_i 
\end{eqnarray}
\begin{eqnarray}
    \alpha_k = \frac{p_k^T r_k}{p_k^T A p_k}
\end{eqnarray}
\begin{eqnarray}
    x_{k+1} = \alpha_k p_k + x_k
\end{eqnarray}

### $\alpha_k$や$p_k,x_k$の漸化式の意味

#### $\alpha_k$
$\alpha_k$は，$k+1$回目のイテレーションで$f(x)$を最小化する係数である．
\begin{eqnarray}
    f(x_{k+1}) = f(x+\alpha_k p_k) = \frac{1}{2}(x+\alpha_k p_k)^T A (x+\alpha_k p_k) - b^T (x+\alpha_k p_k)\\
    \frac{\partial f(x_{k+1})}{\partial \alpha_k} = \alpha_k p_k^T A p_k + p_k^T A x_k - p_k^T b
\end{eqnarray}
なので，$f(x_{k+1})$を最適化するような$\alpha_k$は，
\begin{eqnarray}
    \frac{\partial f(x_{k+1})}{\partial \alpha_k} = 0\\
    \therefore \alpha_k = \frac{p_k^T (b-A x_k)}{p_k^T A p_k} = \frac{p_k^T r_k}{p_k^T A p_k}
\end{eqnarray}

#### $p_k$
共役法では，求めたい解$x$を$A$に対して互いに共役なベクトルの線形結合で表しその結合係数を計算することで，解を計算する．共役勾配法も共役法と同様な計算を行っている．
$p_k$と$p_j(j<k)$が$A$に対して互いに共役であることは次の計算により確かめることができる．
\begin{eqnarray}
    p_k^T A p_j = \left(r_k  - \sum_{i=0}^{k-1} \frac{p_i^T A r_k}{p_i A p_i} p_i \right)^T A p_i = r_k^T Ap_j - \frac{p_j^T A r_k}{p_j^T A p_j}p_j^TAp_j = 0
\end{eqnarray}

#### $x_k$の漸化式
$\alpha_k$を$x_k$に足して$x$を更新することで，$f(x_k)$の最小の値を計算することができる．$\alpha_k$をその都度のコスト関数の最適化によって求めることができる．
> $f(x_k)$が単調減少する証明は存在する？→はい，$a_k=0$にすれば$f(x_{k+1})=f(x_k)$とすることができるので，$f(x_{k+1})\leq f(x_k)$である．また$f(x)$は放物面なので$f(x)$が最適化されれば，必然的に$x$は厳密解に近づくと考えられる．

### 注釈
> 文字の定義の仕方
> 英語大文字が行列，英語小文字がベクトル，ギリシャ文字がスカラーである．

## 実装

In [26]:
import numpy as np

In [27]:
def conjugate_gradient(mat_A,vec_b):
    """
    :param mat_A: `np.ndarray` matrix A
    :param vec_b: `np.ndarray` vector b
    :return: `np.ndarray` solution x
    """
    size = len(vec_b)
    vec_r = np.zeros(size)
    vec_x = np.zeros(size)
    list_vec_p = []
    for k in range(size):
        vec_r = vec_b-mat_A@vec_x
        vec_d = np.zeros(len(mat_A))
        for i in range(k):
            vec_d += (list_vec_p[i] @ mat_A @ vec_r)/(list_vec_p[i] @ mat_A @ list_vec_p[i]) * list_vec_p[i]
        vec_p = vec_r - vec_d
        alpha = (vec_p @ vec_r)/(vec_p@mat_A@vec_p)
        vec_x = alpha*vec_p + vec_x
        list_vec_p.append(vec_p)
        #print("r",vec_r)
        #print("x",vec_x)
        #print("d",vec_d)
        #print(alpha)
    return vec_x

In [28]:
A1 = np.array([[4,1],[1,3]])
A2 = np.array([[2,5,3],[5,3,3],[3,3,3]])
b1 = np.array([2,1])
b2 = np.array([2,-1,3])
print(f"Exact: {np.linalg.inv(A1) @ b1}")
print(f"Conjugate Gradient: {conjugate_gradient(mat_A=A1,vec_b=b1)}")

print(f"Exact: {np.linalg.inv(A2) @ b2}")
print(f"Conjugate Gradient: {conjugate_gradient(mat_A=A2,vec_b=b2)}")

## 条件数
print(f"Condition Number of A1: {np.linalg.cond(A1)}")
print(f"Eigs of A1: {np.linalg.eig(A1)[0]}")

print(f"Condition Number of A2: {np.linalg.cond(A2)}")
print(f"Eigs of A2: {np.linalg.eig(A2)[0]}")

Exact: [0.45454545 0.18181818]
Conjugate Gradient: [0.45454545 0.18181818]
Exact: [-2.  -1.5  4.5]
Conjugate Gradient: [-2.  -1.5  4.5]
Condition Number of A1: 1.938748901931751
Eigs of A1: [4.61803399 2.38196601]
Condition Number of A2: 21.424394502595273
Eigs of A2: [10.06695818 -2.53684115  0.46988297]


## より賢い実装(解説と証明工事中)
先ほどの実装では$\vec{p}$を更新する際の$\vec{d}$の計算で一回のイテレーションで，$O(N^3)$の計算コストが必要とし，イテレーションを$O(N)$繰り返すとするとアルゴリズムトータルでは$O(N^4)$の計算コストである．これはガウスの消去法よりも遅い．そこで$\vec{p}$を更新する部分の高速化を検討する．

In [29]:
def conjugate_gradient_rev(mat_A,vec_b):
    size = len(vec_b)
    vec_x = np.zeros(len(mat_A))
    vec_r = vec_b - mat_A@vec_x
    vec_p = vec_b - mat_A@vec_x
    for k in range(100):
        r_norm = vec_r@vec_r
        if r_norm<=10**-10:
            return vec_x
        vec_Ap = mat_A@vec_p
        alpha = r_norm/(vec_p@vec_Ap)
        vec_x += alpha*vec_p
        vec_rn = vec_r-alpha*vec_Ap
        beta = (vec_rn@vec_rn)/r_norm
        vec_r -= alpha*vec_Ap
        vec_p = vec_r + beta*vec_p
    return vec_x


In [30]:
A1 = np.array([[4,1],[1,3]])
A2 = np.array([[2,1,3],[1,-6,2],[3,2,2]])
b1 = np.array([2,1])
b2 = np.array([2,-1,3])
print(f"Exact: {np.linalg.inv(A1) @ b1}")
print(f"Conjugate Gradient: {conjugate_gradient_rev(mat_A=A1,vec_b=b1)}")

print(f"Exact: {np.linalg.inv(A2) @ b2}")
print(f"Conjugate Gradient: {conjugate_gradient_rev(mat_A=A2,vec_b=b2)}")

## 条件数
print(f"Condition Number of A1: {np.linalg.cond(A1)}")
print(f"Eigs of A1: {np.linalg.eig(A1)[0]}")

print(f"Condition Number of A2: {np.linalg.cond(A2)}")
print(f"Eigs of A2: {np.linalg.eig(A2)[0]}")

Exact: [0.45454545 0.18181818]
Conjugate Gradient: [0.45454545 0.18181818]
Exact: [0.75   0.3125 0.0625]
Conjugate Gradient: [0.75   0.3125 0.0625]
Condition Number of A1: 1.938748901931751
Eigs of A1: [4.61803399 2.38196601]
Condition Number of A2: 7.089377080166892
Eigs of A2: [ 5.39754782 -0.91447682 -6.483071  ]


## 時間の比較

In [31]:
import time

start = time.time()

print(conjugate_gradient(mat_A=A2,vec_b=b2))

t1 = time.time()

print(conjugate_gradient_rev(mat_A=A2,vec_b=b2))

t2 = time.time()

print(t1-start,t2-t1)

[0.75   0.3125 0.0625]
[0.75   0.3125 0.0625]
0.0010027885437011719 0.0


In [32]:
import time

t0 = time.perf_counter()

print(conjugate_gradient(mat_A=A2,vec_b=b2))

t1 = time.perf_counter()

print(conjugate_gradient_rev(mat_A=A2,vec_b=b2))

t2 = time.perf_counter()

print(t1-t0,t2-t1)

[0.75   0.3125 0.0625]
[0.75   0.3125 0.0625]
0.0006790000043110922 0.0003077000001212582


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
import time

def time_measure(func):
    t0 = time.perf_counter()
    func()
    t1 = time.perf_counter()
    return t1-t0

In [ ]:
print(time_measure(conjugate_gradient(mat_A=A2,vec_b=b2)))